# Welcome to DRpy (derpy)

Here is a simple notebook to help you get going with plotting GPM-DPR data. To see how to download your favorite file, see the other example notebook named: "Example_io_DRpy.ipynb"



<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/dopplerchase/DRpy/blob/master/Example_Plotting_DRpy.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/dopplerchase/DRpy/blob/master/Example__DRpy.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [ ]:
#@title Install Required Packages: proplot,cartopy,drpy
%%capture
!pip -q uninstall shapely --yes
!pip -q install shapely cartopy --no-binary shapely --no-binary cartopy
!pip -q install git+https://github.com/dopplerchase/DRpy.git
!pip -q install proplot
!pip -q install pyproj 

In [1]:
#@title Import Required Packages
import pandas as pd 
import drpy 
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#@title JAXA NRT INFO BOX 
#@markdown To make things easy, paste the closest jaxa waypoint info here!
jaxa_waypoint = '2021/03/14 04:00:00 Lat:\t48.106 Lon:\t231.148 Orbit No.:\t040003' #@param {type:"string"}
dtime = pd.to_datetime(jaxa_waypoint.split('Lat')[0])
lat_r = float(jaxa_waypoint.split('Lat:\t')[1].split('Lon')[0])
lon_r = float(jaxa_waypoint.split('Lat:\t')[1].split('Lon:\t')[1].split()[0])
if lon_r > 180:
  lon_r = lon_r - 360 
orbit_n = int(jaxa_waypoint.split('Lat:\t')[1].split('Lon:\t')[1].split()[3])

In [4]:
filename = '../example_files/2A.GPM.DPR.V820180723.20200117-S181128-E184127.V06A.RT-H5'

In [6]:
stime = time.time()
dpr = drpy.core.GPMDPR(filename = filename)
etime =time.time()
print('time elapsed {} seconds'.format(etime-stime))

time elapsed 5.700988054275513 seconds


## Take a look at the xarray datatset (.xrds)

The primary reason for this package was to make an xarray dataset (xrds) so that the effcient methods of xarray could be used easily. 

To see whats in the xrds, simply print out the following

In [7]:
dpr.xrds

<xarray.Dataset>
Dimensions:                (along_track: 2571, cross_track: 25, range: 176)
Coordinates:
    lons                   (along_track, cross_track) float32 -71.04 ... 71.56
    lats                   (along_track, cross_track) float32 21.04 ... 35.99
    time                   (along_track, cross_track) datetime64[ns] 2020-01-...
    alt                    (along_track, cross_track, range) float64 21.7 ......
Dimensions without coordinates: along_track, cross_track, range
Data variables:
    flagSurfaceSnow        (along_track, cross_track) uint8 0 0 0 0 ... 0 0 0 0
    binBBTop               (along_track, cross_track) int16 -1111 ... -1111
    binBBBottom            (along_track, cross_track) int16 -1111 ... -1111
    flagPrecip             (along_track, cross_track) int32 0 0 0 0 ... 0 0 0 0
    typePrecip             (along_track, cross_track) int64 -9223372036854775...
    phaseNearSurface       (along_track, cross_track) int64 -9999 ... -9999
    precipRateNearSurface  (along_track, cross_track) float32 0.0 0.0 ... 0.0
    clutter                (along_track, cross_track, range) int64 0 0 0 ... 1 1
    nearsurfaceKu          (along_track, cross_track) float32 nan nan ... nan
    nearsurfaceKa          (along_track, cross_track) float32 nan nan ... nan
    NSKu_c                 (along_track, cross_track, range) float32 nan ... nan
    epsilon                (along_track, cross_track, range) float32 nan ... nan
    MSKa_c                 (along_track, cross_track, range) float32 nan ... nan
    echotop                (along_track, cross_track, range) int64 0 0 0 ... 0 0
    NSKu                   (along_track, cross_track, range) float32 5.15 ......
    MSKa                   (along_track, cross_track, range) float32 nan ... nan
    R                      (along_track, cross_track, range) float32 0.0 ... nan
    Dm_dpr                 (along_track, cross_track, range) float32 nan ... nan
    Nw_dpr                 (along_track, cross_track, range) float32 nan ... nan

The xrds has the original dataset shape: along_track, cross_track and range

There is a whole bunch of variables I pass along from the original hdf file. But all of them are not here. This is mainly because I have not generalized the code to not take 1 million lines of code and convert all the variables in the file. Eventually, I hope to have a function that properly converts them all. 

If you wish to see the original hdf file, you can still access it in the following way:


In [9]:
list(dpr.hdf)

['AlgorithmRuntimeInfo', 'HS', 'MS', 'NS']

In [ ]:
    import matplotlib.cm as cmx
    import matplotlib.colors as colors 
    import cartopy
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    import matplotlib.ticker as mticker
    from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
    import matplotlib.patheffects as PathEffects
    import pandas as pd 
    from cartopy.mpl.geoaxes import GeoAxes
    from mpl_toolkits.axes_grid1 import AxesGrid
    from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
    import drpy